In [34]:
import os
from typing import List
from langchain.document_loaders import PyPDFLoader, TextLoader
from langchain_community.document_loaders import Docx2txtLoader, UnstructuredWordDocumentLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain_core.documents import Document
import pdfplumber
import logging

# 设置日志
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

def robust_pdf_loader(file_path: str) -> List[Document]:
    """使用 pdfplumber 加载 PDF，处理复杂布局，返回 Document 对象"""
    documents = []
    try:
        with pdfplumber.open(file_path) as pdf:
            text = ""
            for page in pdf.pages:
                page_text = page.extract_text(layout=True)
                if page_text:
                    text += page_text + "\n"
        if text.strip():
            documents.append(Document(page_content=text, metadata={"source": file_path}))
        logging.info(f"pdfplumber 成功加载: {file_path}")
    except Exception as e:
        logging.warning(f"pdfplumber 加载失败: {file_path}, 错误: {e}, 尝试 PyPDFLoader")
        try:
            loader = PyPDFLoader(file_path)
            documents = loader.load()
            logging.info(f"PyPDFLoader 成功加载: {file_path}")
        except Exception as e2:
            logging.error(f"PyPDFLoader 加载失败: {file_path}, 错误: {e2}")
    return documents

def robust_docx_loader(file_path: str) -> List[Document]:
    """尝试多种 DOCX 加载器，返回 Document 对象"""
    loaders = [Docx2txtLoader, UnstructuredWordDocumentLoader]
    for Loader in loaders:
        try:
            loader = Loader(file_path)
            documents = loader.load()
            if documents:
                logging.info(f"{Loader.__name__} 成功加载: {file_path}")
                return documents
        except Exception as e:
            logging.warning(f"{Loader.__name__} 加载失败: {file_path}, 错误: {e}")
    logging.error(f"所有 DOCX 加载器均失败: {file_path}")
    return []

def build_vector_db(knowledge_dir: str, output_dir: str = "./vector_db", model_name: str = "./bge-small-zh-v1.5"):
    """
    从文档目录生成 FAISS 向量库，保存为离线文件。
    :param knowledge_dir: 文档文件夹（PDF/DOC/TXT）
    :param output_dir: 保存路径（FAISS 索引和元数据）
    :param model_name: 嵌入模型（中文优化）
    """
    # 加载文档
    documents = []
    for file in os.listdir(knowledge_dir):
        file_path = os.path.join(knowledge_dir, file)
        ext = os.path.splitext(file)[1].lower()
        try:
            if ext == ".pdf":
                docs = robust_pdf_loader(file_path)
            elif ext in [".doc", ".docx"]:
                docs = robust_docx_loader(file_path)
            elif ext == ".txt":
                loader = TextLoader(file_path, encoding="utf-8")
                docs = loader.load()
            else:
                logging.info(f"跳过不支持格式: {file_path}")
                continue
            documents.extend(docs)
            logging.info(f"成功加载: {file_path}")
        except Exception as e:
            logging.error(f"加载 {file_path} 失败: {e}")
    
    if not documents:
        raise ValueError("没有成功加载任何文档！")
    logging.info(f"共加载 {len(documents)} 个文档")

    # 分块（语义分割）
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=300,
        chunk_overlap=50,
        separators=["。", "！", "？", "\n\n"],
        length_function=len
    )
    chunks = text_splitter.split_documents(documents)
    logging.info(f"生成 {len(chunks)} 个 chunk")

    # 生成嵌入
    try:
        embeddings = HuggingFaceEmbeddings(
            model_name=model_name,
            model_kwargs={"device": "cpu"}  # 改 "cuda" 若有 GPU
        )
        logging.info(f"成功加载嵌入模型: {model_name}")
    except Exception as e:
        logging.warning(f"加载本地模型 {model_name} 失败: {e}, 尝试在线模型")
        model_name = "./bge-small-zh-v1.5"
        embeddings = HuggingFaceEmbeddings(
            model_name=model_name,
            model_kwargs={"device": "cpu"}
        )
        logging.info(f"成功加载在线模型: {model_name}")

    # 建 FAISS 索引（cosine 相似度）
    vectorstore = FAISS.from_documents(chunks, embeddings)
    
    # 保存
    os.makedirs(output_dir, exist_ok=True)
    vectorstore.save_local(output_dir)
    logging.info(f"向量库已保存到 {output_dir}/index.faiss 和 index.pkl")
    return vectorstore

# 示例调用（提前一天运行）
try:
    vector_store = build_vector_db("知识文档")
except Exception as e:
    logging.error(f"构建向量库失败: {e}")

2025-10-18 00:04:45,854 - WARNING - Cannot set gray non-stroke color because /'P144' is an invalid float value
2025-10-18 00:04:45,860 - WARNING - Cannot set gray non-stroke color because /'P146' is an invalid float value
2025-10-18 00:04:45,866 - WARNING - Cannot set gray non-stroke color because /'P148' is an invalid float value
2025-10-18 00:04:45,873 - WARNING - Cannot set gray non-stroke color because /'P150' is an invalid float value
2025-10-18 00:05:19,966 - INFO - pdfplumber 成功加载: 知识文档\02-中国银联全渠道支付平台-第2部分 产品接口规范-产品2 互联网支付跳转支付——B2B产品.pdf
2025-10-18 00:05:19,967 - INFO - 成功加载: 知识文档\02-中国银联全渠道支付平台-第2部分 产品接口规范-产品2 互联网支付跳转支付——B2B产品.pdf
2025-10-18 00:05:42,390 - INFO - pdfplumber 成功加载: 知识文档\09-中国银联全渠道支付平台-第2部分 产品接口规范-产品9 代收产品.pdf
2025-10-18 00:05:42,392 - INFO - 成功加载: 知识文档\09-中国银联全渠道支付平台-第2部分 产品接口规范-产品9 代收产品.pdf
2025-10-18 00:05:42,406 - INFO - Docx2txtLoader 成功加载: 知识文档\1--中国银行业监督管理委员会关于印发农村信用社省（自治区直辖市）联合社管理暂行规定的通知银监发200314号.docx
2025-10-18 00:05:42,407 - INFO - 成功加载: 知识文档\1--中国银行业监督

In [28]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from sentence_transformers import CrossEncoder
from langchain_community.embeddings import SentenceTransformerEmbeddings
import numpy as np

def load_and_query(query: str, vector_db_dir: str = "./vector_db", model_name: str = "./bge-small-zh-v1.5", cross_model_name: str = "./mmarco-mMiniLMv2-L12-H384-v1", top_k: int = 5):
    """
    加载离线向量库，查询并 rerank。
    :param query: 查询语句
    :param vector_db_dir: FAISS 路径
    :param model_name: 嵌入模型
    :param cross_model_name: rerank 模型
    :param top_k: 返回 top-K 结果
    """
    # 加载 FAISS
    embeddings = HuggingFaceEmbeddings(model_name=model_name, model_kwargs={"device": "cpu"})
    vectorstore = FAISS.load_local(vector_db_dir, embeddings, allow_dangerous_deserialization=True)
    
    # 粗召回（k=20）
    retriever = vectorstore.as_retriever(search_kwargs={"k": 20})
    docs = retriever.invoke(query)
    
    # Rerank
    cross_encoder = CrossEncoder(cross_model_name)
    pairs = [[query, doc.page_content] for doc in docs]
    scores = cross_encoder.predict(pairs)
    
    # 按分数排序，取 top-K
    top_indices = np.argsort(scores)[::-1][:top_k]
    top_docs = [docs[i] for i in top_indices]
    top_scores = scores[top_indices]
    
    return top_docs, top_scores


In [29]:
import requests
import os
from typing import List
def call_deepseek_api(query: str, context: str, api_key: str) -> str:
    """
    调用 DeepSeek API，基于查询和 Top-K chunks 生成答案
    """
    # 构造上下文：将 Top-K chunks 拼接为上下文
    prompt = f"""
    你是一个专业的金融领域助手。基于以下上下文，回答用户的问题。确保只给出对应的答案，让答案最准确、简洁。

    问题: {query}

    上下文:
    {context}

    请提供详细的答案。
    """
    
    headers = {
        "Authorization": f"Bearer {api_key}",
        "Content-Type": "application/json"
    }
    
    payload = {
        "model": "deepseek-chat",
        "messages": [
            {"role": "system", "content": "You are a helpful assistant in the financial domain."},
            {"role": "user", "content": prompt}
        ],
        "max_tokens": 500,
        "temperature": 0
    }
    
    try:
        response = requests.post("https://api.deepseek.com/v1/chat/completions", 
                               json=payload, headers=headers)
        response.raise_for_status()
        result = response.json()
        return result['choices'][0]['message']['content']
    except requests.RequestException as e:
        return f"调用 DeepSeek API 失败: {str(e)}"


In [35]:
# 示例查询（现场运行）
query = "瑞银集团因法国税务历史遗留问题支付的总金额转换成人民币是多少元？"
top_docs, scores = load_and_query(query)
print(f"Top-{len(top_docs)} 结果：")
for i, (doc, score) in enumerate(zip(top_docs, scores)):
    print(f"排名 {i+1} (分数: {score:.3f}): {doc.page_content}")

2025-10-18 01:03:19,465 - INFO - Load pretrained SentenceTransformer: ./bge-small-zh-v1.5
2025-10-18 01:03:22,607 - INFO - Use pytorch device: cpu


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Top-5 结果：
排名 1 (分数: 6.322): 。
瑞银集团向法国政府支付7.3亿欧元罚款及1.05亿欧元赔偿金
瑞银集团宣布，已就法国税务历史遗留问题与法国政府达成和解，支付款项总额为8.35亿欧元，其中包括7.3亿欧元罚款及1.05亿欧元赔偿金。
★
市场数据
★
创业板指探底回升涨0.21%，半导体板块尾盘爆发
9月23日，市场探底回升，创业板尾盘快速翻红，此前一度跌逾2%。沪深两市成交额2.49万亿元，较上一个交易日放量3729亿元。板块方面，港口航运、银行等板块涨幅居前，旅游、华为、小金属等板块跌幅居前。截至收盘，沪指跌0.18%，深证成指跌0.29%，创业板指涨0.21%
排名 2 (分数: -5.793): 。                            
                                                    其他交易类型的交易金额的币                 
                                                    种均为人民币，以分为单位，                 
                                                    前 补 0 ， 如  12.34 元 填          
                                                    000000001234
排名 3 (分数: -6.089): 。                          
                                    文                                             
            交易金额        TransAmount M      交易金额的币种均为人民币,保留两位小                     
                                           数，示例：消费金额为      100 元，则交易金额            
                                         

In [36]:
context = "\n\n".join([f"Chunk {i+1}: {doc.page_content}" for i, doc in enumerate(top_docs)])
print(context)

Chunk 1: 。
瑞银集团向法国政府支付7.3亿欧元罚款及1.05亿欧元赔偿金
瑞银集团宣布，已就法国税务历史遗留问题与法国政府达成和解，支付款项总额为8.35亿欧元，其中包括7.3亿欧元罚款及1.05亿欧元赔偿金。
★
市场数据
★
创业板指探底回升涨0.21%，半导体板块尾盘爆发
9月23日，市场探底回升，创业板尾盘快速翻红，此前一度跌逾2%。沪深两市成交额2.49万亿元，较上一个交易日放量3729亿元。板块方面，港口航运、银行等板块涨幅居前，旅游、华为、小金属等板块跌幅居前。截至收盘，沪指跌0.18%，深证成指跌0.29%，创业板指涨0.21%

Chunk 2: 。                            
                                                    其他交易类型的交易金额的币                 
                                                    种均为人民币，以分为单位，                 
                                                    前 补 0 ， 如  12.34 元 填          
                                                    000000001234

Chunk 3: 。                          
                                    文                                             
            交易金额        TransAmount M      交易金额的币种均为人民币,保留两位小                     
                                           数，示例：消费金额为      100 元，则交易金额            
                                           应为00000010000

Chunk 4: 20

							

In [37]:
# 调用 DeepSeek API 生成答案
deepseek_api_key = "sk-afabddf5ba0c4c96abb3567aa3324605"
answer = call_deepseek_api(query, context, deepseek_api_key)
#print(reranked_chunks)
print("DeepSeek API 生成的答案：")
print(answer)

DeepSeek API 生成的答案：
调用 DeepSeek API 失败: ('Connection aborted.', ConnectionResetError(10054, '远程主机强迫关闭了一个现有的连接。', None, 10054, None))
